### Importing Libraries and Data Retrieval

In [1]:
import requests
import pandas as pd
from io import BytesIO
from zipfile import ZipFile

In [2]:
zip_url = "https://archive.ics.uci.edu/static/public/352/online+retail.zip"

# Define headers
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:123.0) Gecko/20100101 Firefox/123.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "Referer": "https://archive.ics.uci.edu/",
    "Upgrade-Insecure-Requests": "1",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-User": "?1",
    "TE": "trailers"
}

# Define cookies
cookies = {
    "_ga_K3WHJGBFH8": "GS1.1.1710294011.6.0.1710294032.0.0.0",
    "_ga": "GA1.2.2124358671.1709592817",
    "_gcl_au": "1.1.496912670.1709592817",
    "_ga_KPV3WXGBTL": "GS1.1.1710294010.5.0.1710294010.0.0.0",
    "__stripe_mid": "21519bdc-5b88-4f4c-bd4f-29e41341f166c8b2cb",
    "_gid": "GA1.2.924643912.1710294012",
    "_gat": "1",
    "__stripe_sid": "2f73d8a4-6ca5-4831-b66f-893ed74184d87696ab"
}

# Read the Excel file directly from the zip archive
with ZipFile(BytesIO(requests.get(zip_url, headers=headers, cookies=cookies).content)) as zf:
    excel_file_name = zf.namelist()[0]
    final_df = pd.read_excel(
        zf.open(excel_file_name),
        engine="openpyxl")  
    

# print first 10 rows:
print(final_df.head(10).to_markdown)


<bound method DataFrame.to_markdown of   InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   
5    536365     22752         SET 7 BABUSHKA NESTING BOXES         2   
6    536365     21730    GLASS STAR FROSTED T-LIGHT HOLDER         6   
7    536366     22633               HAND WARMER UNION JACK         6   
8    536366     22632            HAND WARMER RED POLKA DOT         6   
9    536367     84879        ASSORTED COLOUR BIRD ORNAMENT        32   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850

In [3]:
final_df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [4]:
features = ['CustomerID', 'InvoiceNo', 'InvoiceDate', 'Quantity', 'UnitPrice']
data = final_df[features].copy()
data['TotalSales'] = data['Quantity'] * data['UnitPrice']

In [5]:
#Remove customers with negative values, Eg: customers with returns
data = data[data['TotalSales'] > 0]

### Data Preprocessing and Cleaning

In [6]:
#Identifying null values
pd.DataFrame(zip(data.isnull().sum(), data.isnull().sum()/len(data)), columns=['Count', 'Proportion'], index=data.columns)

,Count,Proportion
CustomerID,132220,0.249423
InvoiceNo,0,0.000000
InvoiceDate,0,0.000000
Quantity,0,0.000000
UnitPrice,0,0.000000
TotalSales,0,0.000000


In [7]:
data = data[pd.notnull(data['CustomerID'])]

In [8]:
# Removing Outliers from the data
def remove_outlier(df, cols):
    for col in cols:
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        low_bound = q1 - 1.5 * iqr
        upp_bound = q3 + 1.5 * iqr  # Corrected this line
        df = df[(df[col] >= low_bound) & (df[col] <= upp_bound)]
    return df
# specify particular columns to remove outliers
cols_to_check = ['Quantity', 'UnitPrice']

# calling the function
clvdata = remove_outlier(data, cols_to_check)


### Using BG/NBD Model to predict the future transactions of each customer

#### R (Recency): Measures how recently a customer made a purchase or engaged with the business, indicating their level of activity or engagement.

#### F (Frequency): Indicates how often a customer makes purchases or interacts with the business, reflecting their buying habits and loyalty.

#### M (Monetary): Reflects the amount of money spent by a customer on purchases, highlighting their value to the business in terms of revenue generation.

#### T (Tenure): Represents the duration of the customer's relationship with the business, measuring their loyalty and retention over time.

In [9]:
import lifetimes
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter

In [10]:
cltv = lifetimes.utils.summary_data_from_transaction_data(clvdata, 'CustomerID', 'InvoiceDate', 'TotalSales' )
cltv = cltv.reset_index()
cltv.head()

,CustomerID,frequency,recency,T,monetary_value
0,12347.0,6.0,365.0,367.0,443.973333
1,12348.0,2.0,110.0,358.0,18.700000
2,12349.0,0.0,0.0,18.0,0.000000
3,12350.0,0.0,0.0,310.0,0.000000
4,12352.0,6.0,260.0,296.0,150.423333


In [11]:
cltv = cltv[cltv['monetary_value'] > 0]

In [12]:
bgf = lifetimes.BetaGeoFitter(penalizer_coef=0.0)
bgf.fit(cltv['frequency'], cltv['recency'], cltv['T'])

<lifetimes.BetaGeoFitter: fitted with 2656 subjects, a: 0.00, alpha: 119.10, b: 0.04, r: 2.04>

#### Predicting the top 10 future transactions for the next week based on historical data

In [13]:
t = 7
cltv['predicting_transactions'] = round(bgf.conditional_expected_number_of_purchases_up_to_time(t, cltv['frequency'], cltv['recency'], cltv['T']),2)
cltv.sort_values(by='predicting_transactions', ascending=False).head(10).reset_index()

,index,CustomerID,frequency,recency,T,monetary_value,predicting_transactions
0,1828,14911.0,129.0,372.0,373.0,657.349612,1.86
1,3885,17841.0,111.0,372.0,373.0,289.742703,1.61
2,323,12748.0,109.0,373.0,373.0,193.042936,1.58
3,2113,15311.0,89.0,373.0,373.0,194.881910,1.29
4,1614,14606.0,87.0,372.0,373.0,93.068851,1.27
5,466,12971.0,63.0,369.0,372.0,84.570317,0.93
6,542,13089.0,58.0,364.0,368.0,605.829828,0.86
7,1559,14527.0,53.0,367.0,369.0,78.524717,0.79
8,1911,15039.0,41.0,359.0,368.0,333.801463,0.62
9,3867,17811.0,37.0,329.0,333.0,149.428919,0.60


In [14]:
# Modeling the monetary value using Gamma-Gamma Model
ggf = GammaGammaFitter(penalizer_coef=0.01)

ggf.fit(cltv['frequency'], cltv['monetary_value'])

<lifetimes.GammaGammaFitter: fitted with 2656 subjects, p: 3.79, q: 0.38, v: 3.69>

In [15]:
cltv['expected_average_sales'] = ggf.conditional_expected_average_profit(cltv['frequency'],
                                       cltv['monetary_value'])


#### Predict top 10 expected average sales for the next week based on historical data

In [16]:
cltv.sort_values(by='expected_average_sales', ascending=False).head(10).reset_index()

,index,CustomerID,frequency,recency,T,monetary_value,predicting_transactions,expected_average_sales
0,3890,17850.0,1.0,1.0,373.0,3182.320000,0.01,3808.892179
1,538,13081.0,7.0,360.0,371.0,2352.595714,0.13,2409.411202
2,1251,14096.0,16.0,97.0,101.0,2250.573125,0.57,2274.046347
3,46,12406.0,1.0,161.0,183.0,1664.450000,0.07,1994.268090
4,3378,17107.0,4.0,245.0,278.0,1639.582500,0.11,1710.432789
5,2810,16303.0,2.0,308.0,333.0,1567.875000,0.06,1709.499245
6,2449,15786.0,1.0,68.0,114.0,1350.660000,0.08,1619.129974
7,872,13555.0,1.0,128.0,201.0,1349.510000,0.06,1617.755141
8,2121,15321.0,1.0,221.0,291.0,1307.120000,0.05,1567.077603
9,742,13365.0,1.0,184.0,217.0,1230.770000,0.06,1475.800649


#### Predicting CLV by Using BG-NBD and Gamma-Gamma Model
##### Predicting the TOP 10 customers' lifetime values expected  in the next month

In [19]:
cltv['predicted_clv'] = ggf.customer_lifetime_value(bgf,
                                                               cltv['frequency'],
                                                               cltv['recency'],
                                                               cltv['T'],
                                                               cltv['monetary_value'],
                                                               time=30,     
                                                               freq='D'    
                                                               ) 
cltv.sort_values(by='predicted_clv', ascending=False).head(10).reset_index()

,index,CustomerID,frequency,recency,T,monetary_value,predicting_transactions,expected_average_sales,predicted_clv,segment
0,1251,14096.0,16.0,97.0,101.0,2250.573125,0.57,2274.046347,143917.668225,A
1,1828,14911.0,129.0,372.0,373.0,657.349612,1.86,658.212609,135525.408468,A
2,542,13089.0,58.0,364.0,368.0,605.829828,0.86,607.606565,57903.432174,A
3,3885,17841.0,111.0,372.0,373.0,289.742703,1.61,290.203462,51544.351003,A
4,1392,14298.0,23.0,352.0,360.0,979.278696,0.37,986.452838,39850.821053,A
5,538,13081.0,7.0,360.0,371.0,2352.595714,0.13,2409.411202,34326.378245,A
6,323,12748.0,109.0,373.0,373.0,193.042936,1.58,193.366864,33737.230450,A
7,1293,14156.0,34.0,320.0,329.0,538.019412,0.56,540.728644,33616.562967,A
8,1247,14088.0,10.0,312.0,322.0,1358.172000,0.19,1381.126819,29123.611196,A
9,2113,15311.0,89.0,373.0,373.0,194.881910,1.29,195.282150,27934.160332,A


#### Segmenting cutsomers based on their CLV

In [23]:
cltv['segment'] = pd.qcut(cltv['predicted_clv'],4,['Newcomer', 'Regular', 'Loyal', 'VIP'])

cltv

,CustomerID,frequency,recency,T,monetary_value,predicting_transactions,expected_average_sales,predicted_clv,segment
0,12347.0,6.0,365.0,367.0,443.973333,0.12,457.044778,5838.124598,VIP
1,12348.0,2.0,110.0,358.0,18.700000,0.06,22.372308,140.678251,Newcomer
4,12352.0,6.0,260.0,296.0,150.423333,0.14,155.269658,2321.176201,Regular
8,12356.0,1.0,80.0,325.0,267.860000,0.03,324.635081,1045.689188,Newcomer
10,12358.0,1.0,149.0,150.0,509.160000,0.07,613.110901,5048.633409,VIP
...,...,...,...,...,...,...,...,...,...
4180,18272.0,5.0,244.0,246.0,446.004000,0.13,461.846793,6873.684006,VIP
4181,18273.0,1.0,255.0,257.0,51.000000,0.05,65.377451,385.323438,Newcomer
4188,18282.0,1.0,119.0,126.0,52.340000,0.08,66.979430,602.871121,Newcomer
4189,18283.0,13.0,334.0,337.0,149.502308,0.23,151.694094,3865.382082,Loyal
